In [1]:
import pandas as pd
import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import fontManager

# 改style要在改font之前
# plt.style.use('seaborn')  

fontManager.addfont('TaipeiSansTCBeta-Regular.ttf')
mpl.rc('font', family='Taipei Sans TC Beta')

In [2]:
data = pd.read_csv('training.csv')

In [82]:
df = data.copy()
df.columns = [
    '交易序號','授權日期','授權時間','顧客ID','交易卡號',
'交易類別','交易型態','特店代號','收單行代碼','mcc_code','交易金額-台幣','網路交易註記','是否分期交易','分期期數','是否紅利交易','實付金額','消費地國別','消費城市','狀態碼',
'超註記碼','Fallback註記','支付型態','消費地幣別','消費地金額','3D交易註記','盜刷與否'
]

# Drop features

In [5]:
df.isnull().sum()

交易序號                0
授權日期                0
授權時間                0
顧客ID                0
交易卡號                0
交易類別                0
交易型態           203455
特店代號                0
收單行代碼               0
mcc_code         4550
交易金額-台幣             0
網路交易註記              0
是否分期交易              0
分期期數                0
是否紅利交易              0
實付金額                0
消費地國別             600
消費城市           266066
狀態碼           8665195
超註記碼                0
Fallback註記          0
支付型態           286656
消費地幣別          498657
消費地金額               0
3D交易註記              0
盜刷與否                0
dtype: int64

In [85]:
df = df.drop(columns=['消費地幣別', '狀態碼', '消費地金額', '實付金額'])

# 序列特徵

## 過去交易金額的mean

In [94]:
df = df.sort_values(by=['授權日期','授權時間'])
groupby_card_mean = df.sort_values(by=['授權日期','授權時間']).groupby(['顧客ID', '交易卡號'])['交易金額-台幣'].expanding().mean().reset_index(level=0, drop=True)
groupby_card_mean = groupby_card_mean.reset_index().set_index('level_1')['交易金額-台幣'].rename('過去交易金額平均')
df = pd.merge(df, groupby_card_mean, left_index=True, right_index=True)
del groupby_card_mean
df.loc[:,'過去交易金額平均'].describe()

count    8.688526e+06
mean     1.426057e+03
std      6.095352e+03
min      0.000000e+00
25%      3.186422e+02
50%      6.138788e+02
75%      1.231684e+03
max      3.016827e+06
Name: 過去交易金額平均, dtype: float64

## 該筆交易金額是否大於那張卡過去交易金額的q75 + 1.5IQR

In [95]:
q75 = df.groupby(['顧客ID', '交易卡號'])['交易金額-台幣'].expanding().quantile(0.75).reset_index()
q75 = q75.set_index('level_2')['交易金額-台幣'].rename('q75')
IQR = (df.groupby(['顧客ID', '交易卡號'])['交易金額-台幣'].expanding().quantile(0.75) - df.groupby(['顧客ID', '交易卡號'])['交易金額-台幣'].expanding().quantile(0.25)).reset_index()
IQR = IQR.set_index('level_2')['交易金額-台幣'].rename('iqr')
df = pd.merge(df, q75, left_index=True, right_index=True, how='left')
df = pd.merge(df, IQR, left_index=True, right_index=True, how='left')
del IQR, q75
df['是否大於過去交易金額的1.5IQR'] = ((df['交易金額-台幣']) >= 1.5 * df['iqr'] + df['q75']).astype(int)
df.loc[:, '是否大於過去交易金額的1.5IQR'].describe()

count    8.688526e+06
mean     1.762526e-01
std      3.810349e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: 是否大於過去交易金額的1.5IQR, dtype: float64

In [9]:
df[['盜刷與否', '過去交易金額平均',  '是否大於過去交易金額的1.5IQR']].corr()

,盜刷與否,過去交易金額平均,是否大於過去交易金額的1.5IQR
盜刷與否,1.000000,0.031786,0.038193
過去交易金額平均,0.031786,1.000000,0.008079
是否大於過去交易金額的1.5IQR,0.038193,0.008079,1.000000


# Preprocessing

In [18]:
df.describe()

,授權日期,授權時間,交易類別,交易型態,mcc_code,交易金額-台幣,網路交易註記,是否分期交易,分期期數,是否紅利交易,...,超註記碼,Fallback註記,支付型態,消費地幣別,消費地金額,3D交易註記,盜刷與否,在外消費,過去交易金額平均,iqr
count,8.688526e+06,8.688526e+06,8.688526e+06,8.485071e+06,8.683976e+06,8.688526e+06,8.688526e+06,8.688526e+06,8.688526e+06,8.688526e+06,...,8.688526e+06,8.688526e+06,8.401870e+06,8.189869e+06,8.688526e+06,8.688526e+06,8.688526e+06,8.688526e+06,8.688526e+06,8.688526e+06
mean,2.750020e+01,1.430832e+05,4.905415e+00,4.714078e+00,3.173557e+02,1.418676e+03,5.719845e-01,1.058661e-02,6.073458e-02,1.272713e-03,...,9.741814e-03,3.058056e-04,5.351585e+00,6.963586e+01,2.806845e+03,8.530687e-02,3.686356e-03,9.897375e-02,1.426057e+03,8.687822e+02
std,1.617132e+01,5.380955e+04,4.744539e-01,2.259769e+00,6.865512e+01,9.518751e+03,4.947911e-01,1.023452e-01,7.347819e-01,3.565240e-02,...,9.821869e-02,1.748463e-02,1.868967e+00,4.386718e+00,1.084921e+05,2.793378e-01,6.060336e-02,2.986268e-01,6.095352e+03,4.193778e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.300000e+01,1.042070e+05,5.000000e+00,4.000000e+00,2.760000e+02,1.050000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,6.000000e+00,7.000000e+01,9.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,3.186422e+02,1.217500e+02
50%,2.700000e+01,1.452280e+05,5.000000e+00,5.000000e+00,2.890000e+02,3.150000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,6.000000e+00,7.000000e+01,2.890000e+02,0.000000e+00,0.000000e+00,0.000000e+00,6.138788e+02,3.510000e+02
75%,4.100000e+01,1.848040e+05,5.000000e+00,5.000000e+00,3.240000e+02,9.000000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,6.000000e+00,7.000000e+01,9.800000e+02,0.000000e+00,0.000000e+00,0.000000e+00,1.231684e+03,7.880000e+02
max,5.500000e+01,2.359590e+05,6.000000e+00,1.000000e+01,5.010000e+02,5.051970e+06,1.000000e+00,1.000000e+00,3.000000e+01,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+01,8.500000e+01,7.473480e+07,1.000000e+00,1.000000e+00,1.000000e+00,3.016827e+06,2.525985e+06


In [110]:
df.columns

Index(['交易序號', '授權日期', '授權時間', '顧客ID', '交易卡號', '交易類別', '交易型態', '特店代號', '收單行代碼',
       'mcc_code', '交易金額-台幣', '網路交易註記', '是否分期交易', '分期期數', '是否紅利交易', '消費地國別',
       '消費城市', '超註記碼', 'Fallback註記', '支付型態', '3D交易註記', '盜刷與否', '過去交易金額平均',
       'q75', 'iqr', '是否大於過去交易金額的1.5IQR'],
      dtype='object')

In [120]:
id_cols = ['交易序號', '顧客ID', '交易卡號']
datetime_cols = ['授權日期', '授權時間']
cate_cols = ['交易類別', '交易型態', 'mcc_code', '消費地國別', '消費城市' ,'支付型態', '特店代號', '收單行代碼']
bi_cols = ['網路交易註記','是否分期交易', '是否紅利交易', '超註記碼', 'Fallback註記', '3D交易註記', '是否大於過去交易金額的1.5IQR']
num_cols = ['交易金額-台幣', '分期期數', '過去交易金額平均']
label_col = ['盜刷與否']

## Drop掉mcc_code, 消費地國別, 消費城市出現次數小於10次的類別

In [105]:
for col in ['mcc_code', '消費地國別', '消費城市' , '特店代號', '收單行代碼']:
    # 計算每個值的出現次數
    value_counts = df[col].value_counts()

    # 找出出現次數小於10的值
    mask = df['mcc_code'].isin(value_counts[value_counts < 10].index)

    # 將符合條件的值設置為 pd.NA
    df.loc[mask, 'mcc_code'] = 'other'

In [106]:
# Convert categorical columns to strings
df[cate_cols] = df[cate_cols].astype(str)
# Use pd.get_dummies with sparse=True to one-hot encode
df_cate = pd.get_dummies(df[cate_cols], sparse=True)
df_cate

,交易類別_0,交易類別_1,交易類別_2,交易類別_3,交易類別_4,交易類別_5,交易類別_6,交易型態_0.0,交易型態_1.0,交易型態_10.0,...,收單行代碼_ff96f75d26ddea0a24e0d5403243f8d1570d96b05ad474ef347c0ae88087ccd0,收單行代碼_ff9968b6cf3874de7d767b94661735cc1d21e54bf2582830ac10b84299aefa12,收單行代碼_ffa3418b34684ea63fb983e1031c24b0fe46a1ecf968eadbd6cc45cdc01c73b1,收單行代碼_ffaa0f221b223e2632913c5cedb7cb54f9023dd481b2c87984ae34336bdae502,收單行代碼_ffb61ec650c8427005120ad438a35e3e8830a6ea448061c27ce0438e3f705eea,收單行代碼_ffba12c94df6213daf9d19a2d04b3d7ac7d9788bf597218a9cf4b74c87f89cab,收單行代碼_ffd949310c78f7de18f56abfbdf76415d1f07980ca16fef72ffcdb425a7f45f7,收單行代碼_ffe19e7ce26f92c0b2a9955fa0204eff0160df9f2ca48b99e3df88d2d07ae309,收單行代碼_ffe23f73f3da4ee77d3d32bf63344a9fade079aa28de102d317b694da2b92f30,收單行代碼_ffe3ab26f013c96af52f14efaece6ea32f7d6500198d2e98113d226d3be436bd
8513276,False,False,False,False,False,True,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4668186,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
415479,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1337039,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1622716,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087361,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
85577,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7158052,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4290596,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [121]:
df3 = pd.concat([df[datetime_cols], df[num_cols], df[bi_cols], df_cate, df[label_col]], axis=1)
df3

,授權日期,授權時間,交易金額-台幣,分期期數,過去交易金額平均,網路交易註記,是否分期交易,是否紅利交易,超註記碼,Fallback註記,...,收單行代碼_ff9968b6cf3874de7d767b94661735cc1d21e54bf2582830ac10b84299aefa12,收單行代碼_ffa3418b34684ea63fb983e1031c24b0fe46a1ecf968eadbd6cc45cdc01c73b1,收單行代碼_ffaa0f221b223e2632913c5cedb7cb54f9023dd481b2c87984ae34336bdae502,收單行代碼_ffb61ec650c8427005120ad438a35e3e8830a6ea448061c27ce0438e3f705eea,收單行代碼_ffba12c94df6213daf9d19a2d04b3d7ac7d9788bf597218a9cf4b74c87f89cab,收單行代碼_ffd949310c78f7de18f56abfbdf76415d1f07980ca16fef72ffcdb425a7f45f7,收單行代碼_ffe19e7ce26f92c0b2a9955fa0204eff0160df9f2ca48b99e3df88d2d07ae309,收單行代碼_ffe23f73f3da4ee77d3d32bf63344a9fade079aa28de102d317b694da2b92f30,收單行代碼_ffe3ab26f013c96af52f14efaece6ea32f7d6500198d2e98113d226d3be436bd,盜刷與否
8513276,0,0,1324.0,0.0,1324.000000,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
4668186,0,1,1360.0,0.0,1360.000000,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
415479,0,2,3266.0,0.0,3266.000000,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
1337039,0,2,1385.0,0.0,1385.000000,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
1622716,0,3,898.0,0.0,898.000000,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8087361,55,235954,20.0,0.0,609.222222,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
85577,55,235955,15.0,0.0,630.702128,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
7158052,55,235957,85.0,0.0,179.403509,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0
4290596,55,235959,530.0,0.0,379.857143,1,0,0,0,0,...,False,False,False,False,False,False,False,False,False,0


In [ ]:
df3.to_csv('after_clean.csv')